# Training a Diffusion Model with W&B

In this notebooks we will instrument the training of a diffusion model with W&B. We will use the Lab3 notebook and add:
- Logging of the training loss
- Sampling from the model during training and logging the samples to W&B
- Saving the model checkpoints to W&B

In [1]:
from pathlib import Path
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from utilities import *

import wandb

In [2]:
# wandb.login() # uncomment if you want to login to wandb

## Setting Things Up

In [3]:
# hyperparameters
num_samples = 32

# diffusion hyperparameters
timesteps = 500
beta1 = 1e-4
beta2 = 0.02

# network hyperparameters
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 16 # 16x16 image
data_dir = Path('./data/')
save_dir = Path('./data/weights/')
save_dir.mkdir(exist_ok=True, parents=True)


# training hyperparameters
batch_size = 100
n_epoch = 32
lrate=1e-3

# we are storing the parameters in a dictionary to be logged to wandb
config = dict(
    num_samples=num_samples,
    timesteps=timesteps,
    beta1=beta1,
    beta2=beta2,
    device=device,
    n_feat=n_feat,
    n_cfeat=n_cfeat,
    height=height,
    save_dir=save_dir,
    batch_size=batch_size,
    n_epoch=n_epoch,
    lrate=lrate,
)

All this is the same as the previous notebook, except for the addition of the context vector size n_cfeat. We will use this to condition the diffusion model on a context vector.

In [4]:
# construct DDPM noise schedule
b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
a_t = 1 - b_t
ab_t = torch.cumsum(a_t.log(), dim=0).exp()    
ab_t[0] = 1

In [5]:
# construct model
nn_model = ContextUnet(in_channels=3, n_feat=n_feat, n_cfeat=n_cfeat, height=height).to(device)

In [6]:
# load dataset and construct optimizer
dataset = CustomDataset.from_np(data_dir/"sprites_1788_16x16.npy", data_dir/"sprite_labels_nc_1788_16x16.npy")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)
optim = torch.optim.Adam(nn_model.parameters(), lr=lrate, eps=1e-5)

sprite shape: (89400, 16, 16, 3)
labels shape: (89400, 5)


In [7]:
# helper function: perturbs an image to a specified noise level
def perturb_input(x, t, noise):
    return ab_t.sqrt()[t, None, None, None] * x + (1 - ab_t[t, None, None, None]) * noise

## Sampling

We will need to instrument the sampler to have telemetry on the generated images while training!

In [8]:
# helper function; removes the predicted noise (but adds some noise back in to avoid collapse)
def denoise_add_noise(x, t, pred_noise, z=None):
    if z is None:
        z = torch.randn_like(x)
    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

In [9]:
# sample with context using standard algorithm
# we make a change to the original algorithm to allow for context explicitely (the noises)
@torch.no_grad()
def sample_ddpm_context(samples, context, save_rate=20):
    # array to keep track of generated steps for plotting
    intermediate = [] 
    for i in range(timesteps, 0, -1):
        # reshape time tensor
        t = torch.tensor([i / timesteps])[:, None, None, None].to(device)

        # sample some random noise to inject back in. For i = 1, don't add back in noise
        z = torch.randn_like(samples) if i > 1 else 0

        eps = nn_model(samples, t, c=context)    # predict noise e_(x_t,t, ctx)
        samples = denoise_add_noise(samples, i, eps, z)
        if i % save_rate==0 or i==timesteps or i<8:
            intermediate.append(samples.detach().cpu().numpy())

    intermediate = np.stack(intermediate)
    return samples.clip(-1, 1), intermediate

## Training

we choose a fixed context vector with 6 of each class, this way we know what to expect on the workspace.

In [10]:
# Noise vector
# x_T ~ N(0, 1), sample initial noise
noises = torch.randn(num_samples, 3, height, height).to(device)  

# A fixed context vector to sample from
ctx_vector = F.one_hot(torch.tensor([0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4]), 5).to(device=device).float()

The following training cell takes very long to run on CPU, we have already trained the model for you on a GPU equipped machine.

### You can visit the result of this [training here](https://wandb.ai/deeplearning-ai-temp/sprite_diffusion/runs/lqf74fua)

In [11]:
# same code as before, added comments on the extra W&B instrumentation lines
# create a wandb run
run = wandb.init(project="sprite_diffusion", job_type="train", anonymous="allow", config=config)

for ep in tqdm(range(n_epoch), leave=True, total=n_epoch):
    # set into train mode
    nn_model.train()
    optim.param_groups[0]['lr'] = lrate*(1-ep/n_epoch)
    
    pbar = tqdm(dataloader, leave=False)
    for x, c in pbar:   # x: images  c: context
        optim.zero_grad()
        x = x.to(device)
        c = c.to(x)   
        context_mask = torch.bernoulli(torch.zeros(c.shape[0]) + 0.8).to(device)
        c = c * context_mask.unsqueeze(-1)        
        noise = torch.randn_like(x)
        t = torch.randint(1, timesteps + 1, (x.shape[0],)).to(device) 
        x_pert = perturb_input(x, t, noise)      
        pred_noise = nn_model(x_pert, t / timesteps, c=c)      
        loss = F.mse_loss(pred_noise, noise)
        loss.backward()    
        optim.step()

        # we log the relevant metrics to the workspace
        wandb.log({"loss": loss.item(),
                   "lr": optim.param_groups[0]['lr'],
                   "epoch": ep})

    # save model periodically
    if ep%4==0 or ep == int(n_epoch-1):
        nn_model.eval()
        ckpt_file = save_dir/f"context_model.pth"
        torch.save(nn_model.state_dict(), ckpt_file)

        # save model to wandb as an Artifact
        artifact_name = f"{wandb.run.id}_context_model"
        at = wandb.Artifact(artifact_name, type="model", 
                            metadata={"loss":loss.item(), "epoch":ep})
        at.add_file(ckpt_file)
        wandb.log_artifact(at, aliases=[f"epoch_{ep}"])

        # sample the model and log the images to W&B
        samples, _ = sample_ddpm_context(noises, ctx_vector[:num_samples])
        wandb.log({"train_samples": [wandb.Image(img) for img in samples.split(1)]})

# finish W&B run
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape (deeplearning-ai-temp). Use `wandb login --relogin` to force relogin


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

KeyboardInterrupt: 